In [12]:
#import the module
import EdgePrediction, json
import pandas as pd

In [2]:
from scipy.stats import hypergeom

In [3]:
def setup(files, optimisation, require_all):
    #create a new instance
    ep = EdgePrediction.EdgePrediction()
    
    ep.to_predict = 'risk_gene'
    ep.optimisation_method = optimisation
    ep.require_all_predictors = require_all
    
    for to_load in files:
        print(to_load)
        ep.CSV_to_graph(fname = to_load)
    
    nw_order = list(ep.graphs.keys()).sort()
    ep.network_order = nw_order
    ep.preprocess()
    
    return ep

In [4]:
def hyper_prob_at_least(pop_n, pop_true, draw_n, draw_true):
    #prob of at least h hits is i - cdf(h-1)
    prb = hypergeom.cdf(draw_true-1, pop_n, pop_true, draw_n)
    prb_h_plus = 1 - prb
    return prb_h_plus

In [5]:
def make_row(result, target_name, predictable, n_new_genes_total, verbose=True):
    row = {}
    a = set(result['new_hits'])
    n_validated = len(a.intersection(predictable))
    n_predicted = len(a)
    n_can_validate = len(predictable)
    if n_predicted > 0:
        validation_precision = n_validated / n_predicted
    else:
        validation_precision = 0.0
    validation_recall = n_validated / n_can_validate
    n_genes_total = sum(result['contingency'].values())
    n_genes_predicted_total = result['contingency']['tp'] + result['contingency']['fp']
    n_genes_train = result['contingency']['tp'] + result['contingency']['fn']
    
    keep = ['F1', 'F05', 'F2', 'ACC', 'J', 'PREC', 'REC', 'FDR', 'FPR', 'hits_total', 'hits_new', 'hits_known', 'model_built']
    row = {x:result[x] for x in keep}
    for x in ['tp', 'tn', 'fp', 'fn']:
        row[x] = result['contingency'][x]
    for name, weight in result['weights'].items():
        row['weight_' + name] = weight
    row['target'] = target_name
    
    row['validation_precision'] = validation_precision
    row['validation_recall'] = validation_recall
    row['n_genes_total'] = n_genes_total
    row['n_new_genes_total'] = n_new_genes_total
    row['n_new_genes_predictable'] = n_can_validate
    row['model_built'] = True
    row['n_validated'] = n_validated
    
    #hyper_prob_at_least(pop_n, pop_true, draw_n, draw_true)
    pop_n = result['contingency']['tn'] + result['contingency']['fp']
    pop_true = n_can_validate
    draw_n = n_predicted #this was tp+fp before but should only be fp i.e. new_hits
    draw_true = n_validated
    prob = hyper_prob_at_least(pop_n, pop_true, draw_n, draw_true)
    row['prob'] = prob
    
    if verbose:
        print("model for {} achieved {:.2f} validation rate at {:.2f} validation recall (found {}/{} with pred={})".format(target_name, validation_precision, validation_recall, n_validated, n_can_validate, n_predicted))
        print("predicted {}/{} genes associated with {} training genes".format(n_genes_predicted_total, n_genes_total, n_genes_train))
        print("predicted association rate {:.3f}".format(n_genes_predicted_total/n_genes_total))
        print("{}/{} new associations could have been predicted and were tested".format(n_can_validate, n_new_genes_total))
        print("p hyper {:0.3f}".format(prob))
        print()
    
    return row
    

In [6]:
def run(ep, to_train, valid):
    all_genes = set(ep.graphs['risk_gene']['sourcenodes'])
    rows = []
    n_todo = len(to_train)
    for tgt_idx in range(len(to_train)):
        print("{}/{}".format(tgt_idx, n_todo))
        target_name = to_train[tgt_idx]
        #check if any of the new associations can potentially be predicted
        new_genes = set(valid[target_name])
        predictable = new_genes.intersection(all_genes)
        if len(predictable) == 0:
            print("**** no predictable genes for", target_name)
            row = {'target': target_name, 'n_new_genes_predictable': 0, 'n_new_genes_total': len(new_genes)}
            rows.append(row)
            continue

        result = ep.predict(target = target_name, calculate_auc = True)
        if result['model_built'] == True:
            row = make_row(result, target_name, predictable, len(new_genes))
        else:
            row = {'target': target_name, 'n_new_genes_predictable': len(predictable), 'n_new_genes_total': len(new_genes)}
            row['model_built'] = False
        rows.append(row)
    return rows

In [7]:
files = [
         "train/disgenet_v7.0_train_2018.csv",
         "train/Gene Ontology v2020-11-17.csv",
         "train/IntAct v2020-11-06.csv"
         ]

In [8]:
with open('test/disgenet_new_since_2018.txt') as f:
    valid = json.load(f)

In [9]:
ep = setup(files, optimisation = 'graph_sparse', require_all=False)

train/disgenet_v7.0_train_2018.csv
train/Gene Ontology v2020-11-17.csv
train/IntAct v2020-11-06.csv
8107 source nodes are common to all 3 input graphs
deleting 1002 nodes that don't overlap between networks
deleting another 271 nodes that now have degree zero
deleting 11604 nodes that don't overlap between networks
deleting another 1942 nodes that now have degree zero
deleting 13358 nodes that don't overlap between networks
deleting another 2201 nodes that now have degree zero


In [10]:
possible_targets = set(ep.graphs['risk_gene']['targetnodes'])
print(len(possible_targets), 'diseases in network')

10329 diseases in network


In [11]:
to_train_tmp = list(valid.keys())
to_train = [x for x in to_train_tmp if x in possible_targets]

Train all models. This can take ~ 2 hours.

In [68]:
rows = run(ep, to_train, valid)

0/1131
**** no predictable genes for C0036341
1/1131
ignoring C0002395 as a predictor in network risk_gene
model for C0002395 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=1095)
predicted 1182.0/8107.0 genes associated with 88 training genes
predicted association rate 0.146
1/1 new associations could have been predicted and were tested
p hyper 0.137

2/1131
ignoring C0007102 as a predictor in network risk_gene
model for C0007102 achieved 0.00 validation rate at 0.33 validation recall (found 1/3 with pred=1160)
predicted 1274.0/8107.0 genes associated with 147 training genes
predicted association rate 0.157
3/5 new associations could have been predicted and were tested
p hyper 0.377

3/1131
ignoring C0009375 as a predictor in network risk_gene
model for C0009375 achieved 0.00 validation rate at 0.50 validation recall (found 2/4 with pred=1009)
predicted 1113.0/8107.0 genes associated with 140 training genes
predicted association rate 0.137
4/5 new associat

ignoring C0006142 as a predictor in network risk_gene
model for C0006142 achieved 0.01 validation rate at 0.13 validation recall (found 19/144 with pred=1522)
predicted 2048.0/8107.0 genes associated with 667 training genes
predicted association rate 0.253
144/344 new associations could have been predicted and were tested
p hyper 0.992

30/1131
ignoring C0011999 as a predictor in network risk_gene
model for C0011999 achieved 0.03 validation rate at 0.33 validation recall (found 1/3 with pred=29)
predicted 52.0/8107.0 genes associated with 23 training genes
predicted association rate 0.006
3/3 new associations could have been predicted and were tested
p hyper 0.011

31/1131
ignoring C0019193 as a predictor in network risk_gene
model for C0019193 achieved 0.00 validation rate at 0.75 validation recall (found 3/4 with pred=1563)
predicted 1833.0/8107.0 genes associated with 270 training genes
predicted association rate 0.226
4/4 new associations could have been predicted and were tested
p

ignoring C0393770 as a predictor in network risk_gene
model for C0393770 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=0)
predicted 4.0/8107.0 genes associated with 4 training genes
predicted association rate 0.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

55/1131
ignoring C0494410 as a predictor in network risk_gene
model for C0494410 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=0)
predicted 4.0/8107.0 genes associated with 4 training genes
predicted association rate 0.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

56/1131
ignoring C0751758 as a predictor in network risk_gene
model for C0751758 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=0)
predicted 4.0/8107.0 genes associated with 4 training genes
predicted association rate 0.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

57/1131
igno

ignoring C4721507 as a predictor in network risk_gene
model for C4721507 achieved 0.01 validation rate at 0.67 validation recall (found 2/3 with pred=374)
predicted 433.0/8107.0 genes associated with 67 training genes
predicted association rate 0.053
3/4 new associations could have been predicted and were tested
p hyper 0.006

81/1131
**** no predictable genes for C0004134
82/1131
ignoring C0032300 as a predictor in network risk_gene
model for C0032300 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=362)
predicted 413.0/8107.0 genes associated with 52 training genes
predicted association rate 0.051
1/1 new associations could have been predicted and were tested
p hyper 1.000

83/1131
ignoring C0235874 as a predictor in network risk_gene
model for C0235874 achieved 0.00 validation rate at 1.00 validation recall (found 2/2 with pred=1382)
predicted 1513.0/8107.0 genes associated with 151 training genes
predicted association rate 0.187
2/2 new associations coul

ignoring C0028754 as a predictor in network risk_gene
model for C0028754 achieved 0.00 validation rate at 0.14 validation recall (found 1/7 with pred=1114)
predicted 1247.0/8107.0 genes associated with 187 training genes
predicted association rate 0.154
7/7 new associations could have been predicted and were tested
p hyper 0.654

109/1131
ignoring C0043094 as a predictor in network risk_gene
model for C0043094 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=1879)
predicted 1939.0/8107.0 genes associated with 97 training genes
predicted association rate 0.239
1/1 new associations could have been predicted and were tested
p hyper 1.000

110/1131
ignoring C0005586 as a predictor in network risk_gene
model for C0005586 achieved 0.00 validation rate at 0.50 validation recall (found 5/10 with pred=2369)
predicted 2725.0/8107.0 genes associated with 422 training genes
predicted association rate 0.336
10/18 new associations could have been predicted and were tested

...

ignoring C1847319 as a predictor in network risk_gene
model for C1847319 achieved 0.00 validation rate at 0.00 validation recall (found 0/2 with pred=0)
predicted 3.0/8107.0 genes associated with 3 training genes
predicted association rate 0.000
2/3 new associations could have been predicted and were tested
p hyper 1.000

990/1131
ignoring C1861848 as a predictor in network risk_gene
model for C1861848 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

991/1131
ignoring C1864827 as a predictor in network risk_gene
model for C1864827 achieved 0.08 validation rate at 1.00 validation recall (found 1/1 with pred=12)
predicted 14.0/8107.0 genes associated with 2 training genes
predicted association rate 0.002
1/1 new associations could have been predicted and were tested
p hyper 0.001

99

ignoring C1838244 as a predictor in network risk_gene
model for C1838244 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

1019/1131
ignoring C1720958 as a predictor in network risk_gene
model for C1720958 achieved 0.00 validation rate at 1.00 validation recall (found 2/2 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
2/2 new associations could have been predicted and were tested
p hyper 1.000

1020/1131
ignoring C1852372 as a predictor in network risk_gene
model for C1852372 achieved 0.67 validation rate at 0.40 validation recall (found 2/5 with pred=3)
predicted 5.0/8107.0 genes associated with 3 training genes
predicted association rate 0.001
5/5 new associations could have been predicted and were tested
p hyper 0.0

ignoring C0751602 as a predictor in network risk_gene
model for C0751602 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=0)
predicted 4.0/8107.0 genes associated with 4 training genes
predicted association rate 0.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

1049/1131
ignoring C0271934 as a predictor in network risk_gene
model for C0271934 achieved 0.00 validation rate at 1.00 validation recall (found 2/2 with pred=8105)
predicted 8107.0/8107.0 genes associated with 2 training genes
predicted association rate 1.000
2/3 new associations could have been predicted and were tested
p hyper 1.000

1050/1131
ignoring C0266617 as a predictor in network risk_gene
model for C0266617 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
1/1 new associations could have been predicted and were tested
p hyper 1.0

ignoring C1865872 as a predictor in network risk_gene
model for C1865872 achieved 0.00 validation rate at 0.00 validation recall (found 0/1 with pred=44)
predicted 50.0/8107.0 genes associated with 7 training genes
predicted association rate 0.006
1/1 new associations could have been predicted and were tested
p hyper 1.000

1078/1131
ignoring C0265215 as a predictor in network risk_gene
model for C0265215 achieved 0.05 validation rate at 1.00 validation recall (found 2/2 with pred=42)
predicted 54.0/8107.0 genes associated with 12 training genes
predicted association rate 0.007
2/2 new associations could have been predicted and were tested
p hyper 0.000

1079/1131
ignoring C3714506 as a predictor in network risk_gene
model for C3714506 achieved 0.01 validation rate at 0.33 validation recall (found 1/3 with pred=98)
predicted 112.0/8107.0 genes associated with 14 training genes
predicted association rate 0.014
3/3 new associations could have been predicted and were tested
p hyper 0.036


ignoring C2931208 as a predictor in network risk_gene
model for C2931208 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

1105/1131
ignoring C1859353 as a predictor in network risk_gene
model for C1859353 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
1/1 new associations could have been predicted and were tested
p hyper 1.000

1106/1131
ignoring C1861355 as a predictor in network risk_gene
model for C1861355 achieved 0.00 validation rate at 1.00 validation recall (found 1/1 with pred=8106)
predicted 8107.0/8107.0 genes associated with 1 training genes
predicted association rate 1.000
1/1 new associations could have been predicted and were tested
p hyp

In [69]:
df = pd.DataFrame(rows)

In [70]:
df.head()

,target,n_new_genes_predictable,n_new_genes_total,F1,F05,F2,ACC,J,PREC,REC,...,fp,fn,weight_risk_gene,weight_go_annotation,weight_encoded_PPI,validation_precision,validation_recall,n_genes_total,n_validated,prob
0,C0036341,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C0002395,1,1,0.137008,0.090324,0.283572,0.864808,0.852086,0.073604,0.988636,...,1095.0,1.0,0.1,0.0,0.0,0.000913,1.000000,8107.0,1.0,0.136551
2,C0007102,3,5,0.160450,0.108716,0.306122,0.852843,0.629782,0.089482,0.775510,...,1160.0,33.0,0.4,0.0,0.6,0.000862,0.333333,8107.0,1.0,0.376610
3,C0009375,4,5,0.166002,0.113240,0.310819,0.871099,0.616210,0.093441,0.742857,...,1009.0,36.0,0.8,0.0,0.9,0.001982,0.500000,8107.0,2.0,0.080713
4,C0011265,3,4,0.132597,0.087209,0.276498,0.864438,0.863019,0.071006,1.000000,...,1099.0,0.0,0.1,0.0,0.0,0.002730,1.000000,8107.0,3.0,0.002564


In [71]:
df.to_csv("temporal_validation_gs_nra.csv") #graph_sparse, not require all